In [1]:
using Random
using ScikitLearn 
@sk_import svm:SVR
using ScikitLearn.GridSearch: GridSearchCV
using ScikitLearn: fit!, predict
using ScikitLearn
@sk_import metrics:mean_squared_error
import Distributions: Uniform 
using Distributions
using LinearAlgebra
using MLDataUtils
using PyCall

In [2]:
using Plots

In [129]:
using CSV
using DataFrames

In [3]:
function random_simplex(dim, M)
    t = rand(1:M, dim-1)
    t = append!(t,M)
    t_sort = sort(t)
    p = [t_sort[i+1]- t_sort[i] for i in 1:(dim-1)]
    p = append!(p,t_sort[1])
    shuffle!(p)/M
end
   

function opt_solution_oracle(prob_vec)
    p_plus = prob_vec[1]
    p_nega = prob_vec[2]
    out = min(log(p_plus/p_nega), 10)
    out = max(out, -10)
end

function softmax(x)
#     max = maximum(x)
#     e_x = collect(exp(elem -max) for elem in x)
#     e_x/sum(e_x)
    exp.(x)./sum(exp.(x))
end

function cost_func(w,z)
    log(1+exp(-z*w))
end

function grad_cost_func(w,z)
    -z/(1+ exp(z*w))
end

grad_cost_func (generic function with 1 method)

# Generate samples and fit the approximate oracle

In [4]:
k =2 # dim of prob
d = 1 # dim of solution
n1 = 1000
p_samples = collect(random_simplex(2, n1) for i in 1:n1)
w_samples = collect(opt_solution_oracle(p) for p in p_samples);

idx = shuffle(1:n1)
idx1 = idx[1:Int(0.8*n1)]
idx2 = idx[Int(0.8*n1)+1:end]
p_train = p_samples[idx1]
w_train = w_samples[idx1]
p_test = p_samples[idx2]
w_test = w_samples[idx2];

In [6]:
py"""
from sklearn.svm import SVR
import numpy as np
import numdifftools as nd
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.metrics import mean_squared_error
def fit_sol_map_SVR(df_p_train, df_w_train, df_p_test, df_w_test):
#     X = df_p_train.values
    X = df_p_train
#     y1 = df_w_train['w1'].values
    y1 = df_w_train
    svr = GridSearchCV(SVR(kernel='rbf'), cv=5,
                   param_grid={"C": [1e-1, 1e0, 1e1, 1e2],
                               "gamma": np.logspace(-2, 2, 10)})
    svr_model1 = svr.fit(X, y1)
    w1_pred = svr.predict(df_p_test)
    print('w1 Mean squared error:', mean_squared_error(df_w_test, w1_pred))
    return svr_model1

def approx_sol_oracle(prob_test, model1, grad=True):
    dim_prob = 2
    fun1 = lambda x: model1.predict(x.reshape(1, dim_prob)).reshape(-1,1)
    w1 = fun1(prob_test)
    w = np.array([w1]).reshape(-1,1)
    if grad == True:
        grad1 = nd.Gradient(fun1)(np.array([[list(prob_test)]]))
        gradw = np.array([grad1])
        return w, gradw
    else:
        return w
def approx_sol_oracle2(prob_test, model1, grad=False):
    dim_prob = 2
    fun1 = lambda x: model1.predict(x.reshape(1, dim_prob)).reshape(-1,1)
    w1 = fun1(prob_test)
    w = np.array([w1]).reshape(-1,1)
    if grad == True:
        grad1 = nd.Gradient(fun1)(np.array([[list(prob_test)]]))
        gradw = np.array([grad1])
        return w, gradw
    else:
        return w
"""

In [7]:
svr1 = py"fit_sol_map_SVR"(p_train, w_train, p_test, w_test)

PyObject GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1.0, 10.0, 100.0],
                         'gamma': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])})

## true value of B

In [8]:
Random.seed!(123)
# B_true = rand(0:5,2,6)
B_true = [4  1  0  3  2  5
 2  1  4  4  2  1]

2×6 Array{Int64,2}:
 4  1  0  3  2  5
 2  1  4  4  2  1

# useful functions for nonparametric prescriptive benchmarks 

In [9]:
function grad_prob(B,x)
    temp = 1/(exp(dot(B[1,:]-B[2,:],x)) + exp(dot(B[2,:]-B[1,:],x)))
    [[temp*x, -temp*x], [-temp*x, temp*x]]
end

grad_prob (generic function with 1 method)

In [10]:
function get_batch_index(sample_size, batch_size)
    rand(1:sample_size, batch_size)
end

function get_knn_count(x, x_train,z_train, k)
    num_x = size(x_train)[1]
    dist_list = collect(norm(x - x_train[i,:]) for i in 1:num_x )  
        idx = sortperm(dist_list)[1:k]
        vec = z_train[idx]
        n_plus = length(vec[vec.>0])
        n_minus = length(vec[vec.<0])
        return n_plus, n_minus
end

    function get_kernel_count(x,x_train,z_train, h)
        num_x = size(x_train)[1]
        weight = collect(exp(-(norm(x - x_train[i,:])/h)^2) for i in 1:num_x)
        n_plus = 0
        n_minus = 0
        for i in 1:num_x
            if z_train[i] >0
                n_plus += weight[i]
            else
                n_minus += weight[i]
            end
        end
        return n_plus, n_minus
end 
    

get_kernel_count (generic function with 1 method)

In [11]:
py"""
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
def get_count_tree(x,x_train, z_train, dep =3 ):
    clf = DecisionTreeClassifier(random_state=0, max_depth = dep)
    clf.fit(x_train,z_train)
    train_nodes = clf.apply(x_train)
    node = clf.apply([x])
    index = np.where(train_nodes == node[0])
    n_plus = 0
    n_minus = 0
#     print(index)
    for ind in index[0]:
        if z_train[ind] > 0:
            n_plus += 1
#             print("plus")
        elif z_train[ind] < 0:
            n_minus += 1
#             print("minus")
    return n_plus, n_minus
# def get_count_rf(x,x_train, z_train, dep =2 ):
#     clf = RandomForestClassifier(random_state=0, max_depth = dep)
#     clf.fit(x_train,z_train)
#     train_nodes = clf.apply(x_train)
#     node = clf.apply([x])
#     index = np.where(train_nodes == node[0])
#     n_plus = 0
#     n_minus = 0
# #     print(index)
#     for ind in index[0]:
#         if z_train[ind] > 0:
#             n_plus += 1
# #             print("plus")
#         elif z_train[ind] < 0:
#             n_minus += 1
# #             print("minus")
#     return n_plus, n_minus
"""

# Generate samples

In [12]:
p = 5
M = 1
num_x0 = 5000
num_t = 2000
num = num_x0 + num_t

7000

In [34]:
x_samples = M*rand(Uniform(0,1), (num,p)); #x without intercept term

In [35]:
x_samples1 = zeros((num,p+1)); #x with intercept term

In [36]:
z_samples = zeros(num)
for i in 1:num
    x_t = append!(x_samples[i,:],1)
    x_samples1[i,:] = x_t
    p_true = softmax(B_true*x_t)
#     println(p_true)
    bern = 2*rand(Bernoulli(p_true[1]),1)[1] - 1
    z_samples[i]= bern    
end   

[0.963205675946752, 0.03679432405324794]
[0.8793990421319782, 0.12060095786802182]
[0.9023120357392462, 0.09768796426075367]
[0.9238910648139094, 0.07610893518609065]
[0.5850753793492612, 0.41492462065073865]
[0.8529636540177036, 0.1470363459822964]
[0.9355307426737441, 0.06446925732625584]
[0.9724880003837815, 0.027511999616218495]
[0.9873513897876895, 0.012648610212310427]
[0.9197850152569323, 0.08021498474306765]
[0.932767118570597, 0.06723288142940291]
[0.9908263327062099, 0.009173667293790178]
[0.9243900535289932, 0.07560994647100687]
[0.9536312001603112, 0.04636879983968877]
[0.8840204736176492, 0.11597952638235076]
[0.9753740561503856, 0.024625943849614394]
[0.9906049814293711, 0.009395018570628862]
[0.8869030051189486, 0.11309699488105142]
[0.9707968956222536, 0.02920310437774635]
[0.9565522622660054, 0.043447737733994504]
[0.9766194363146324, 0.023380563685367537]
[0.7848293520539229, 0.21517064794607718]
[0.9632181266668639, 0.03678187333313596]
[0.985873799257475, 0.01412620

[0.9380014417116342, 0.06199855828836582]
[0.9305712541685319, 0.06942874583146807]
[0.9707452015063625, 0.029254798493637487]
[0.7933430288479881, 0.20665697115201187]
[0.8554429463757933, 0.14455705362420662]
[0.8709194731893233, 0.12908052681067678]
[0.9472795482646091, 0.05272045173539085]
[0.7394358337626493, 0.2605641662373506]
[0.9823048274793331, 0.01769517252066686]
[0.9471240051391381, 0.05287599486086197]
[0.9603705732698197, 0.03962942673018028]
[0.9906181941608074, 0.009381805839192472]
[0.9142214693448597, 0.08577853065514032]
[0.8927447148491977, 0.10725528515080236]
[0.9602973325495556, 0.03970266745044431]
[0.9064318275272757, 0.09356817247272428]
[0.8889191090817871, 0.11108089091821302]
[0.8936534345272364, 0.10634656547276347]
[0.653347040825315, 0.34665295917468497]
[0.7372613793720044, 0.2627386206279957]
[0.9821171675037631, 0.01788283249623679]
[0.922230697860641, 0.07776930213935898]
[0.9831720954164905, 0.01682790458350941]
[0.8403326518194985, 0.1596673481805

[0.6913420358870417, 0.3086579641129583]
[0.819717083893624, 0.1802829161063761]
[0.9738531295300811, 0.026146870469919065]
[0.9933697450562963, 0.0066302549437037665]
[0.9537971566851623, 0.046202843314837795]
[0.9610625774978088, 0.03893742250219119]
[0.979848290272058, 0.02015170972794201]
[0.9550827267381486, 0.04491727326185133]
[0.9934220865296525, 0.006577913470347547]
[0.7436138171964346, 0.2563861828035653]
[0.9411132564891852, 0.05888674351081483]
[0.8378964986828425, 0.16210350131715756]
[0.9432092129382283, 0.05679078706177167]
[0.9466705763346877, 0.0533294236653123]
[0.8252872347408368, 0.1747127652591633]
[0.9903897235446367, 0.009610276455363217]
[0.9704826827220059, 0.029517317277994105]
[0.9199840372642091, 0.08001596273579083]
[0.9223007025172498, 0.07769929748275019]
[0.7860257230299466, 0.21397427697005353]
[0.9595875973899078, 0.04041240261009221]
[0.8972926210157527, 0.10270737898424731]
[0.8756222725024451, 0.12437772749755488]
[0.8930466951234614, 0.10695330487

[0.5055469410094424, 0.49445305899055764]
[0.8774500436332553, 0.12254995636674464]
[0.9389351494902719, 0.06106485050972816]
[0.9750757542634743, 0.024924245736525758]
[0.865421369983906, 0.13457863001609405]
[0.9494348318248641, 0.050565168175135955]
[0.9935047308521976, 0.0064952691478023855]
[0.9644489405190176, 0.03555105948098241]
[0.979722273424001, 0.02027772657599897]
[0.9760212529847585, 0.02397874701524144]
[0.45473420949066, 0.5452657905093401]
[0.8500706995267131, 0.1499293004732869]
[0.9759505081178872, 0.02404949188211286]
[0.9627051228816959, 0.037294877118304114]
[0.9785013368382227, 0.02149866316177732]
[0.7433551518359761, 0.2566448481640239]
[0.9890493853155865, 0.010950614684413572]
[0.956725622595656, 0.043274377404344117]
[0.7359912475306561, 0.26400875246934374]
[0.8496757032686619, 0.15032429673133801]
[0.9155982975661773, 0.08440170243382272]
[0.9155206335276359, 0.08447936647236398]
[0.8096416377548945, 0.1903583622451055]
[0.9769321492088455, 0.0230678507911

[0.9582857028723457, 0.041714297127654364]
[0.949254576857085, 0.05074542314291501]
[0.9443413619541718, 0.05565863804582819]
[0.9235076714358783, 0.07649232856412172]
[0.9676072988699144, 0.032392701130085605]
[0.8975806773342496, 0.1024193226657504]
[0.787311513461025, 0.21268848653897499]
[0.988172319054557, 0.011827680945442943]
[0.6567136448312194, 0.3432863551687806]
[0.9791095001426963, 0.020890499857303752]
[0.9638810691928418, 0.03611893080715815]
[0.9786143045025006, 0.021385695497499295]
[0.929008121670104, 0.07099187832989592]
[0.7050154961265704, 0.29498450387342956]
[0.6842043630906465, 0.31579563690935347]
[0.7591703463137794, 0.24082965368622047]
[0.832009686183138, 0.16799031381686216]
[0.9609920678096775, 0.03900793219032255]
[0.9610895943457001, 0.03891040565429997]
[0.7010550485606593, 0.29894495143934086]
[0.9489899398777557, 0.05101006012224428]
[0.9409935476726446, 0.059006452327355303]
[0.9722949550346227, 0.027705044965377287]
[0.8778836898983685, 0.12211631010

...

[0.8708718781907359, 0.12912812180926417]
[0.88093602126412, 0.11906397873588008]
[0.8351299654673073, 0.16487003453269267]
[0.8984210959179221, 0.10157890408207798]
[0.8859403570509901, 0.11405964294900998]
[0.9905561906699591, 0.009443809330040942]
[0.8049869258225825, 0.19501307417741748]
[0.8933903489640952, 0.1066096510359047]
[0.968026413292618, 0.03197358670738203]
[0.9896869601511497, 0.010313039848850458]
[0.7148701119094839, 0.2851298880905161]
[0.8343325769711973, 0.1656674230288027]
[0.692941664235759, 0.30705833576424113]
[0.9307279824233483, 0.06927201757665165]
[0.6817716307198657, 0.31822836928013437]
[0.9069490118923532, 0.0930509881076467]
[0.9573207947893727, 0.04267920521062731]
[0.9454384990914058, 0.05456150090859429]
[0.8976265001186667, 0.10237349988133324]
[0.9350649134439861, 0.06493508655601389]
[0.9250475275579552, 0.07495247244204481]
[0.9693365712828346, 0.030663428717165445]
[0.8439458664151199, 0.15605413358488007]
[0.8123223161314291, 0.1876776838685708

[0.8291217928667579, 0.17087820713324212]
[0.8883156855915276, 0.11168431440847237]
[0.9574528699929495, 0.04254713000705044]
[0.9715461187300648, 0.028453881269935227]
[0.9879386339943783, 0.012061366005621615]
[0.9650437433178616, 0.03495625668213844]
[0.8924785652921442, 0.10752143470785587]
[0.8088185847052629, 0.191181415294737]
[0.9294022126412753, 0.07059778735872464]
[0.601583288127858, 0.39841671187214206]
[0.9463456156572535, 0.05365438434274655]
[0.8962107647982613, 0.10378923520173855]
[0.7442102248980408, 0.2557897751019592]
[0.80634660761545, 0.19365339238454995]
[0.8261102118630663, 0.17388978813693365]
[0.9889504855664649, 0.01104951443353499]
[0.9910228063260434, 0.008977193673956592]
[0.9810532171467783, 0.0189467828532218]
[0.9853046142636586, 0.014695385736341523]
[0.9795882212813392, 0.02041177871866078]
[0.9688449084488192, 0.031155091551180854]
[0.8964889974354845, 0.10351100256451558]
[0.8457306216527508, 0.15426937834724921]
[0.4560835481141437, 0.5439164518858

[0.9384362690991259, 0.061563730900874046]
[0.8744481925292937, 0.12555180747070632]
[0.9756511413894465, 0.02434885861055348]
[0.9724458637468522, 0.027554136253147815]
[0.9780769722371793, 0.02192302776282074]
[0.9670917771813209, 0.03290822281867907]
[0.7447012859287807, 0.2552987140712194]
[0.810699389145552, 0.18930061085444805]
[0.8296791682157153, 0.17032083178428475]
[0.9733894763143609, 0.026610523685639174]
[0.9061543231893056, 0.0938456768106945]
[0.9742557924687699, 0.025744207531230072]
[0.7680433410323316, 0.2319566589676684]
[0.829552374289768, 0.17044762571023203]
[0.858118319055993, 0.14188168094400694]
[0.9086749345424158, 0.09132506545758423]
[0.8841773075412808, 0.1158226924587193]
[0.7966639330488907, 0.20333606695110928]
[0.5918567096363204, 0.4081432903636795]
[0.9088008650542845, 0.0911991349457156]
[0.9279071770255125, 0.07209282297448752]
[0.975118318032807, 0.024881681967193113]
[0.8124366190183175, 0.1875633809816824]
[0.9861711212618304, 0.01382887873816974

[0.9833814349339564, 0.01661856506604371]
[0.8935215700159344, 0.10647842998406556]
[0.9356720626923927, 0.06432793730760723]
[0.9317025133744529, 0.06829748662554709]
[0.6827331455488026, 0.3172668544511974]
[0.6974990921830353, 0.3025009078169647]
[0.9746001612245067, 0.025399838775493247]
[0.8745179048400991, 0.1254820951599009]
[0.8664748812506003, 0.13352511874939962]
[0.7996202456782417, 0.2003797543217583]
[0.9664058121440066, 0.03359418785599343]
[0.9824850632582538, 0.017514936741746215]
[0.8986651030147403, 0.10133489698525967]
[0.8450688497540345, 0.15493115024596546]
[0.9903595637149589, 0.009640436285041154]
[0.8583325309449384, 0.14166746905506156]
[0.9942636394208344, 0.0057363605791655925]
[0.834978362457382, 0.16502163754261798]
[0.8501993708193156, 0.14980062918068435]
[0.9730413597367784, 0.026958640263221572]
[0.9134302118104768, 0.0865697881895233]
[0.9583861255351834, 0.0416138744648166]
[0.5911538323751103, 0.4088461676248897]
[0.6504931316600732, 0.3495068683399

[0.8848068424571591, 0.11519315754284089]
[0.5724568677975458, 0.42754313220245427]
[0.9194251096663718, 0.08057489033362827]
[0.9713591284346891, 0.028640871565311]
[0.9765149587188654, 0.023485041281134576]
[0.9594585171509509, 0.040541482849049175]
[0.9950112852400105, 0.004988714759989487]
[0.9542722672355937, 0.04572773276440627]
[0.8647917921397911, 0.13520820786020893]
[0.9924489202538437, 0.007551079746156361]
[0.9506271367968685, 0.04937286320313158]
[0.7508592756360029, 0.24914072436399706]
[0.9900948609122979, 0.009905139087702113]
[0.9840187346460149, 0.01598126535398512]
[0.949124724412126, 0.050875275587874]
[0.8505471955627453, 0.14945280443725475]
[0.34525559618577245, 0.6547444038142275]
[0.7899334943968799, 0.21006650560312004]
[0.8311014588037559, 0.16889854119624412]
[0.9641037181685684, 0.03589628183143163]
[0.9307766646126124, 0.06922333538738758]
[0.9934729742307631, 0.006527025769237062]
[0.931567388635423, 0.06843261136457703]
[0.649899423617587, 0.350100576382

# cvxpy layer benchmark
Tried using pycall for the cvxpy layer benchmark, did not work. Same code works in python. 
Dump csv files of training and test sets. 

In [87]:
#PyCall.pyversion

v"3.7.7"

In [128]:
py"""
import cvxpy as cp
import torch 
from cvxpylayers.torch import CvxpyLayer
import numpy as np
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter

p = 5


def train(iters, Xtrain, ztrain):
    class Net(nn.Module):
        def __init__(self):
          super(Net, self).__init__()
          # First fully connected layer
          self.fc1 = nn.Linear(p+1, 32)
          # Second fully connected layer that outputs 
          self.fc2 = nn.Linear(32, 1)
        def forward(self, x):
            x = nn.LeakyReLU(0.1)(self.fc1(x))
            x = nn.LeakyReLU(0.1)(self.fc2(x))
            return x


    dim0 = ztrain.shape[0]
    ztrain = ztrain.reshape((dim0,1))
    # problem
    w_cvxpy = cp.Variable(1)
    z_cvxpy = cp.Parameter(1)
    objective = cp.logistic(1+ cp.exp(-cp.multiply(z_cvxpy,w_cvxpy))) + 0.001*cp.norm(w_cvxpy)**2 
    problem = cp.Problem(cp.Minimize(objective), [w_cvxpy <= 10, w_cvxpy >= -10 ])
    assert problem.is_dpp()
    net = Net()
    cvxlayer = CvxpyLayer(problem, [z_cvxpy], [w_cvxpy])
    results = []

    optimizer = optim.SGD(net.parameters(), lr=0.01)  
    torch.autograd.set_detect_anomaly(True)
    for i in range(iters):
        out1 = net(Xtrain)
        out, = cvxlayer(out1)
#        loss = torch.mean(torch.log(torch.add(torch.exp(torch.neg(torch.multiply(out,ztrain))), 1)))    
#        optimizer.zero_grad()   # zero the gradient buffers
#        loss.backward()
#        optimizer.step()    # Does the update
#        results.append(loss.item())
#        print("(iter %d) loss: %g " % (i, results[-1]))
    return results, net, cvxlayer

def predict(net, cvxlayer, Xtest, ztest):
    z_pred = net(Xtest)
    w_test, = cvxlayer(z_pred)
    return w_test

def main_cvxlayer(Xtrain, ztrain, Xtest, ztest):
    Xtrain = Xtrain.astype(np.float32)
    ztrain = ztrain.astype(np.float32)
    Xtest = Xtest.astype(np.float32)
    ztest = ztest.astype(np.float32)
    Xtrain, Xtest, ztrain, ztest = map(
        torch.from_numpy, [Xtrain, Xtest, ztrain, ztest])
    results, net, cvxlayer = train(50, Xtrain, ztrain)
#    w_test = predict(net, cvxlayer, Xtest, ztest)
    return net
    

"""

In [116]:
# num_x = 50
# idx3 = rand(1:num_x0, num_x)
# x_train = x_train01[idx3,:]
# z_train = z_train0[idx3,:]
# w_test = py"main_cvxlayer"(x_test1, z_train, x_test1, z_test)
# print(w_test)
# for i in 1:num_t
#     z = z_test[i]
#     w = w_test[i]
#     cost_t = cost_func(w,z)
#     cost_test1 += cost_t
# end  

In [121]:
# Pkg.add("DataFrames")

# Main

In [332]:
idx = shuffle(1:num_x0);

In [333]:
# specify num = training size:
#num = 50
num = 1000

1000

In [334]:

idx1 = idx[1:num]
idx2 = idx[num+1:end];

In [335]:
x_train = x_samples[idx1,:]
x_train1 = x_samples1[idx1, :]
z_train = z_samples[idx1]
x_test = x_samples[idx2,:]
x_test1 = x_samples1[idx2, :]
z_test = z_samples[idx2];

In [336]:
# dump csvs
CSV.write("x_train.csv", Tables.table(x_train1), append = false)

CSV.write("z_train.csv", Tables.table(reshape(z_train, length(z_train), 1)), append = false)

CSV.write("x_test.csv", Tables.table(x_test1), append = false)

CSV.write("z_test.csv", Tables.table(reshape(z_test, length(z_test), 1)), append = false)

"z_test.csv"

# test set performance

## true B

In [337]:
cost_test0 = 0
for i in 1:num_t
    x = append!(x_test[i, :],1)
    z = z_test[i]
    p = softmax(B_true*x)
    w = opt_solution_oracle(p)
    cost_t = cost_func(w[1],z[1])
    cost_test0 += cost_t
end 
cost_test0 = cost_test0/num_t

0.3113324503626359

## Our method

In [488]:
# B = zeros((2,6))
alpha = 0.01
# alpha = 0.1
total_ite = 10 #1000
batch_size = 128
cost_list = zeros(total_ite)
for ite in 1:total_ite
    println("ite ", ite)
    #     println("old B ", B)
    grad = zeros((2,6))
    cost = 0
    b_index = get_batch_index(num_x, batch_size)
    for indx in b_index
        x = append!(x_train[indx, :],1)
    #         print(x)
    #         print(z)
        z = z_train[indx]
        p = softmax(B*x)
        result = py"approx_sol_oracle"(p,svr1)
        w = result[1]
        grad_w = result[2]
    #         println(grad_w)
        grad_c = grad_cost_func(w[1],z[1])
    #         println(grad_c)
        grad_t = grad_c*grad_w
        grad_t =  grad_prob(B,x)[1]*grad_t[1] + grad_prob(B,x)[2]*grad_t[2]
        grad_t = hcat(grad_t[1] , grad_t[2])
        grad += transpose(grad_t)
    end
    grad = grad/batch_size
    #     println("grad ",grad)
    for i in 1:num_x
        x = append!(x_train[i, :],1)
        z = z_train[i]
        p = softmax(B*x)
        result = py"approx_sol_oracle2"(p,svr1)
        w = result[1]
        cost_t = cost_func(w[1],z[1])
        cost += cost_t
    end
    cost = cost/num_x
    println("cost ", cost)
    cost_list[ite] = cost
    B = B - alpha* grad
    #     println("update B ", B)
end

ite 1
cost 0.3494719725879051
ite 2
cost 0.3494153039175525
ite 3
cost 0.34942431410689845
ite 4
cost 0.34946729531846166
ite 5
cost 0.3494968924949367
ite 6
cost 0.3494664351724854
ite 7
cost 0.349490921619074
ite 8
cost 0.34946321466084207
ite 9
cost 0.3494726591163422
ite 10
cost 0.349451665814623


In [489]:
cost_test1 = 0
for i in 1:num_t
    x = append!(x_test[i, :],1)
    z = z_test[i]
    p = softmax(B*x)
    w = opt_solution_oracle(p)
    cost_t = cost_func(w[1],z[1])
    cost_test1 += cost_t
end
cost_test1 = cost_test1/num_t

0.3415587212065811

## cvxpy layer

In [493]:
# load cvxpylayer decisions from csv
w_nn = CSV.read("w_test.csv", datarow=1, DataFrame)
w_nn = convert(Matrix, w_nn);

In [494]:
cost_test2 = 0
for i in 1:num_t
#     print(i)
    z = z_test[i]
    w = w_nn[i]
    cost_t = cost_func(w,z)
    cost_test2 += cost_t
end
cost_test2 = cost_test2/num_t

0.42301068894008304

## nonparametric prescriptive methods

 #### KNN

In [365]:
cost_test3 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, 800)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test3 += cost_t
end 
cost_test3 = cost_test3/num_t

0.38752768359608064

In [366]:
cost_test4 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, 1)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test4 += cost_t
end 
cost_test4 = cost_test4/num_t

0.36834323743953507

In [370]:
cost_test5 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, 5)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test5 += cost_t
end
cost_test5 = cost_test5/num_t

0.38663706822789146

 #### Kernel

In [371]:
cost_test6 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, 0.1)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test6 += cost_t
end 
cost_test6 = cost_test6/num_t

0.6197008834415776

In [372]:
cost_test6 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, 100)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test6 += cost_t
end 
cost_test6 = cost_test6/num_t

0.3875254782893816

In [376]:
cost_test6 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, 5)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test6 += cost_t
end 
cost_test6 = cost_test6/num_t

0.38663706822789146

#### tree

In [377]:
cost_test7 = 0
for i in 1:num_t
    n_p, n_n = py"get_count_tree"(x_test[i,:], x_train, z_train, dep =2)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test7 += cost_t
end 
cost_test7 = cost_test7/num_t

0.3309671714133481

In [380]:
cost_test8 = 0
for i in 1:num_t
    n_p, n_n = py"get_count_tree"(x_test[i,:], x_train, z_train, dep =3)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test8 += cost_t
end 
cost_test8 = cost_test8/num_t

0.3686909393733938

## SAA

In [381]:
cost_test9 = 0
for i in 1:num_t
    n_p, n_n = get_kernel_count(x_test[i,:], x_train, z_train, num_x)
    p = [n_p/(n_p+n_n), n_n/(n_p+n_n)]
    w = opt_solution_oracle(p)
    z = z_test[i]
    cost_t = cost_func(w[1],z[1])
    cost_test9 += cost_t
end 
cost_test9 = cost_test9/num_t

0.387518757765629

# Results

| Algorithm   | N=50        | N=100         | N=1000       | 
| :---        |    :----:   |          ---: |         ---: |
| Optimal     | 0.310       | 0.305         |  0.311       |
| Our method  | 0.343       | 0.329         |  0.341       |
| KNN         | 0.395       | 0.349         |  0.368       |
| kernel      | 0.396       | 0.367         |  0.387       |
| tree        | 1.257       | 0.349         |  0.331       |
| cvxlayer    | 0.436       | 0.406         |  0.423       |
| SAA         | 0.396       | 0.368         |  0.388       |